 We will need to load our DAG file <font color="red">("DAG code name")</font> into the DAG folder <font color=red>(gs://your-env-bucket/dags/)</font>. Before we do this let's set up some variables to make our code cleaner:
- <font color=red> Variables:--projectA /projectB / ?? </font>
     We will need to load our DAG file <font color="red">("DAG code name")</font> into the DAG folder <font color=red>(gs://your-env-bucket/dags/)</font>. Before we do this let's set up some variables to make our code cleaner:
- <font color=red> Variables:--projectA /projectB / ?? </font>

README.md
<h2>Running Dataflow job in project A from Composer in project B</h2>

Being able to use a Composer environment running in project B to launch a Dataflow Job in different projects (Project A i.e).

Basically we want to have a central project from where we use Composer to launch Dataflow jobs to different projects.


In this particular use case we have the following requirements:
<ul>
<li> Project B must have Composer API enabled and Dataflow API disabled.</li>
    <li> Project A must have Composer API disabled and Dataflow API enabled.</li>
<li> We don't want to  make use of Project B SA. So we will create a SA in Project A that will be the one used to launch the DF jobs.</li>
<li><a href=https://airflow.apache.org/integration.html#dataflowpythonoperator>We will use DataflowPythonOperator and its gcp_conn_id parameter </a> </li>
</ul>

STEPS:

Before you begin:

You'll use the two files that are in in this repo "DAG_using_DFPythonOperator.py" and "DAG_using_BashOperator.py". You'll need to do the following changes to those files:

Change <>

1. Create Project B and enable Composer API.
2. Create Composer Environment in project B.
3. Create Project A and enable Dataflow API.
- Create Bucket inside Project A (to place staging,temp and output files from Dataflow Job).

4. Create SA in project A <SA-name@project-id.iam.gserviceaccount.com> and grant it the appropriate permissions so that it can launch a DF job:
<ul>
<li>Compute Admin</li>
<li>Dataflow Admin</li>
<li>Dataflow worker</li>
<li>Storage Object Admin</li>
</ul>

5. Create a keyjson file for your SA <SA-name@project-id.iam.gserviceaccount.com>, download it and keep it. 
6. To use the "gcp_conn_id" parameter in DataflowPythonOperator we need to do the following (https://cloud.google.com/composer/docs/how-to/managing/connections#creating_new_airflow_connections):

- Go to your Airflow UI --> Admin --> Connections
- Click on Create
- Enter the following values:

-- Conn Id: "<name-you'll-use-in-gcp_conn_id-parameter>"

-- Conn Type: Choose Google Cloud Platform

-- Project ID: "your-project-A-ID"
    
-- Keyfile Path or Keyfile JSON: (Either upload the previously generated SA keyfile in step 5 to Composer Cloud Storage bucket path (/home/airflow/gcs/data/<your-dataflow-SA-key.json>) and add the path to Keyfile Path field or add the contents of the file directly to Keyfile JSON).

7. You'll need to load the code file "DAG_using_DFPythonOperator.py" to your DAG folder, but before doing so perform the following changes to the file:
- Change "staging-bucket" with the name of the bucket you created in step 3.
- Change "project-A" to your ProjectA id created in Step 3.
- Load the "wordcount.py" file in this repo to your data/ folder in your Cloud Storage bucket associated to your Cloud Composer environment (run "gcloud composer environments describe "your-cloud-composer-env" --location "your-composer-env-location"" to see your bucket name) 
- Change the gcp_conn_id name to the one you have given to the connection created in step 6.

8. This will not work and it should throw some error like the following:

"IOError: Could not upload to GCS path gs://\\<staging-bucket\\>/.../... access denied. Please verify that credentials are valid and that you have write access to the specified path."

My staging bucket is in ProjectA, and the SA which should be used for the connection has been granted StorageAdmin permissions in that project. Still it doesn't work due to <a href=https://issues.apache.org/jira/browse/AIRFLOW-2009>this issue</a>

9. Although not that neat, we can use the following workaround: 

Only using BashOperator to launch the Job without using the DataflowPythonOperator. To use this approach load the file "DAG_using_BashOperator.py" to your DAG folder. Before doing so do:


- Locate the keyjson file created in step 5 to your data/ folder in your Cloud Storage bucket associated to your Cloud Composer environment (we will need to export our GOOGLE_APPLICATION_CREDENTIALS to this path). Now change "your-project-A-sa.json" to your KeyJson filename.
- Change "staging-bucket" with the name of the bucket you created in step 3.
- Change "project-A" to your ProjectA-id created in Step 3.


In [ ]:
#DAG file:

import datetime
 
from airflow import models
from airflow.contrib.operators.dataflow_operator import DataFlowPythonOperator
from airflow.operators import BashOperator
yesterday = datetime.datetime.combine(
    datetime.datetime.today() - datetime.timedelta(1),
    datetime.datetime.min.time())
 
 
default_args = {
    'start_date':yesterday
}
 
with models.DAG(
    'dataflow_python_gcp_conn_id',
    schedule_interval=None,
    default_args=default_args) as dag:
    
        bash_nothing = BashOperator(task_id='nothing_2',bash_command='echo nothing')
        
        run_dataflow_python = DataFlowPythonOperator(
			task_id='df-conn-gcp-id-from-json',
			py_file='/home/airflow/gcs/data/wordcount.py',
			options={'runner':'DataflowRunner',
                                 'output':'gs://staging-bucket-hijo-project/out',
                                 'temp_location':'gs://staging-bucket-hijo-project/teemp',
                                 'staging_location':'gs://staging-bucket-hijo-project/staging',
                                 'project':'hijo-project'},
                        gcp_conn_id='cloud-dataflow-hijo-project-from-location')
        bash_nothing >> run_dataflow_python